In [7]:
import os
from PIL import Image
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tqdm import tqdm

# Define the path to the directory
directory_path = r"D:\datasets\Underwater_Image\WHOI\archive\dataset_pm\training"

# Initialize empty lists for images and labels
X_train = []
Y_train = []
class_label = 0

# Loop through subdirectories (classes)
for class_folder in tqdm(sorted(os.listdir(directory_path))):
    class_path = os.path.join(directory_path, class_folder)
    
    for image_file in os.listdir(class_path):
        if image_file.endswith('.png'):
            image_path = os.path.join(class_path, image_file)
            
            # Load image, convert to RGB and resize
            img = Image.open(image_path).convert('RGB')
            img = img.resize((32, 32))
            img_array = np.array(img)
            
            # Append image and label to lists
            X_train.append(img_array)
            Y_train.append(class_label)
            
    class_label = class_label + 1

# Convert lists to numpy arrays
X_train = np.array(X_train)
Y_train = np.array(Y_train)

print(f'X_train shape: {X_train.shape}')
print(f'Y_train shape: {Y_train.shape}')

100%|██████████████████████████████████████████████████████████████████████████████████| 19/19 [11:14<00:00, 35.52s/it]

X_train shape: (2872, 32, 32, 3)
Y_train shape: (2872,)


In [8]:
import os
from PIL import Image
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tqdm import tqdm

# Define the path to the directory
directory_path = r"D:\datasets\Underwater_Image\WHOI\archive\dataset_pm\training"

# Initialize empty lists for images and labels
X_val = []
Y_val = []
class_label = 0

# Loop through subdirectories (classes)
for class_folder in tqdm(sorted(os.listdir(directory_path))):
    class_path = os.path.join(directory_path, class_folder)
    
    for image_file in os.listdir(class_path):
        if image_file.endswith('.png'):
            image_path = os.path.join(class_path, image_file)
            
            # Load image, convert to RGB and resize
            img = Image.open(image_path).convert('RGB')
            img = img.resize((32, 32))
            img_array = np.array(img)
            
            # Append image and label to lists
            X_val.append(img_array)
            Y_val.append(class_label)
            
    class_label = class_label + 1

# Convert lists to numpy arrays
X_val = np.array(X_val)
Y_val = np.array(Y_val)

print(f'X_train shape: {X_val.shape}')
print(f'Y_train shape: {Y_val.shape}')

100%|██████████████████████████████████████████████████████████████████████████████████| 19/19 [14:38<00:00, 46.25s/it]

X_train shape: (2872, 32, 32, 3)
Y_train shape: (2872,)


In [ ]:
import os
from PIL import Image
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tqdm import tqdm

# Define the path to the directory
directory_path = r"D:\datasets\Underwater_Image\WHOI\archive\dataset_pm\training"

# Initialize empty lists for images and labels
X_test = []
Y_test = []
class_label = 0

# Loop through subdirectories (classes)
for class_folder in tqdm(sorted(os.listdir(directory_path))):
    class_path = os.path.join(directory_path, class_folder)
    
    for image_file in os.listdir(class_path):
        if image_file.endswith('.png'):
            image_path = os.path.join(class_path, image_file)
            
            # Load image, convert to RGB and resize
            img = Image.open(image_path).convert('RGB')
            img = img.resize((32, 32))
            img_array = np.array(img)
            
            # Append image and label to lists
            X_test.append(img_array)
            Y_test.append(class_label)
            
    class_label = class_label + 1

# Convert lists to numpy arrays
X_test = np.array(X_test)
Y_test = np.array(Y_test)

print(f'X_train shape: {X_test.shape}')
print(f'Y_train shape: {Y_test.shape}')

  0%|                                                                                           | 0/19 [00:00<?, ?it/s]

In [2]:
from tensorflow.keras.utils import to_categorical

# Create the ResNet model without the top (fully connected) layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))    
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(num_classes, activation='softmax'))  # Change units to match the number of classes

model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
# Training the model
epochs = 100
model.fit((X_train,to_categorical(Y_train)), epochs=epochs, validation_data=(X_val,to_categorical(Y_val)))

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

# Make predictions on the test dataset
predictions = model.predict(X_test)

# Convert one-hot encoded predictions back to class labels
predicted_labels = np.argmax(predictions, axis=1)

# Calculate accuracy
accuracy = accuracy_score(Y_test, predicted_labels)

# Calculate F1 score
f1 = f1_score(Y_test, predicted_labels, average='weighted')

print(f"Test Loss: {test_loss}")
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")

In [ ]:
from tensorflow.keras.utils import to_categorical
from tensorflow import keras
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

model = keras.models.load_model(r"F:\Pre-Trained_Models\Pre-Trained_Models\whoi_32_model.h5")

x = model.layers[-4].output  # Access the last 4th layer from the end
x = GlobalAveragePooling2D()(x)
output = Dense(num_classes, activation='softmax')(x)
new_model = keras.models.Model(inputs=model.input, outputs=output)
# Compile the model
new_model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
epochs = 100
new_model.fit((X_train,to_categorical(Y_train)), epochs=epochs, validation_data=(X_val,to_categorical(Y_val)))

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

# Make predictions on the test dataset
predictions = new_model.predict(X_test)

# Convert one-hot encoded predictions back to class labels
predicted_labels = np.argmax(predictions, axis=1)

# Calculate accuracy
accuracy = accuracy_score(Y_test, predicted_labels)

# Calculate F1 score
f1 = f1_score(Y_test, predicted_labels, average='weighted')

print(f"Test Loss: {test_loss}")
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")